In [1]:
# Suppress warnings
import warnings
warnings.simplefilter('ignore')

import pyrcel
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
### From the coefficients CSV
coeffs = pd.read_csv('20220813_coeff.csv')
this_row = coeffs.iloc[0]

In [3]:
### Attempting to reproduce the result from 13 August 2022 12z
N1 = 4630.896855
Dg1 = 25.70563148 / 1000
Kmore1 = 0.25
Kless1 = 0.04
fmore1 = 0.4
K1 = Kmore1 if fmore1 > 0.5 else Kless1
S1 = 1.677524762

N2 = 960.8553055
Dg2 = 58.74768485 / 1000
Kmore2 = 0.25
Kless2 = 0.04
fmore2 = 0.4
K2 = Kmore2 if Dg2 > 0.5 else Kless2
S2 = 1.320694585

N3 = 1417.213727
Dg3 = 118.7624418 / 1000
Kmore3 = 0.26
Kless3 = 0.04
fmore3 = 0.68
K3 = Kmore3 if fmore3 > 0.5 else Kless3
S3 = 1.726582835

N4 = 1.212512221
Dg4 = 800 / 1000
Kmore4 = 0.26
Kless4 = 0.26
fmore4 = 1
K4 = Kmore4 if fmore4 > 0.5 else Kless4
S4 = 1.717141449

In [4]:
N1 = this_row['N1']
Dg1 = this_row['mu1'] / 1000
Kmore1 = this_row['kmore1']
Kless1 = this_row['kless1']
fmore1 = this_row['fmore1']
K1 = Kmore1 if fmore1 > 0.5 else Kless1
S1 = this_row['std1']

N2 = this_row['N2']
Dg2 = this_row['mu2'] / 1000
Kmore2 = this_row['kmore2']
Kless2 = this_row['kless2']
fmore2 = this_row['fmore2']
K2 = Kmore2 if Dg2 > 0.5 else Kless2
S2 = this_row['std2']

N3 = this_row['N3']
Dg3 = this_row['mu3'] / 1000
Kmore3 = this_row['kmore3']
Kless3 = this_row['kless3']
fmore3 = this_row['fmore3']
K3 = Kmore3 if fmore3 > 0.5 else Kless3
S3 = this_row['std3']

N4 = this_row['N4']
Dg4 = this_row['mu4'] / 1000
Kmore4 = this_row['k4']
Kless4 = this_row['k4']
fmore4 = this_row['fmore4']
K4 = Kmore4 if fmore4 > 0.5 else Kless4
S4 = this_row['std4']

In [4]:
print(N1)
print(Dg1/2)
print(S1)
print(K1)

4630.896855
0.01285281574
1.677524762
0.04


In [5]:
print(N2)
print(Dg2/2)
print(S2)
print(K2)

960.8553055
0.029373842424999998
1.320694585
0.04


In [6]:
print(N3)
print(Dg3/2)
print(S3)
print(K3)

1417.213727
0.059381220900000004
1.726582835
0.26


In [7]:
print(N4)
print(Dg4/2)
print(S4)
print(K4)

1.212512221
0.4
1.717141449
0.26


In [8]:


## IDEAS:
# - use multimode distribution?
# - multipy N by fmore if fmore > 0.5 or fless if fmore < 0.5

## mode 1
mode1 = pyrcel.AerosolSpecies('mode1',
                          pyrcel.Lognorm(mu=Dg1/2,
                                     sigma=S1,
                                     N=N1), kappa=K1, bins=100)
## mode 2
mode2 = pyrcel.AerosolSpecies('mode2',
                          pyrcel.Lognorm(mu=Dg2/2,
                                     sigma=S2,
                                     N=N2), kappa=K2, bins=100)
## mode 3
mode3 = pyrcel.AerosolSpecies('mode3',
                          pyrcel.Lognorm(mu=Dg3/2,
                                     sigma=S3,
                                     N=N3), kappa=K3, bins=100)
## mode 4
mode4 = pyrcel.AerosolSpecies('mode4',
                          pyrcel.Lognorm(mu=Dg4/2,
                                     sigma=S4,
                                     N=N4), kappa=K4, bins=100)

In [9]:
aerosol_dict = {
    'mode1' : mode1,
    'mode2' : mode2,
    'mode3' : mode3,
    'mode4' : mode4
}

In [10]:
P0 = 100000. # Pressure, Pa
T0 = 280.    # Temperature, K
S0 = .1/100     # Supersaturation
w0 = 1     # vertical velocity, m/s

In [11]:
mod = pyrcel.ParcelModel([mode1, mode2, mode3, mode4], w0, T0, -0.02, P0, console=False, accom=1)

In [12]:
ptrace, aer_out = mod.run(100/w0, 1/w0, solver='cvode', output='dateframes', terminate=True)

Could not find cannot import name 'dopri5' from 'assimulo.lib' (/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/assimulo/lib/__init__.py)
Could not find cannot import name 'rodas' from 'assimulo.lib' (/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/assimulo/lib/__init__.py)
Could not find cannot import name 'odassl' from 'assimulo.lib' (/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/assimulo/lib/__init__.py)
Could not find ODEPACK functions.
Could not find RADAR5
Could not find GLIMDA.


[WARNING][rank 0][/Users/runner/miniforge3/conda-bld/sundials-split_1719463006984/work/sundials/src/cvodes/cvodes.c:3555][CVode] At the end of the first step, there are still some root functions identically 0. This warning will not be issued again.


In [13]:
n = len(ptrace['S']) - 1
activated_amounts = []

In [14]:
for mode_key, mode_value in aerosol_dict.items():
    f, unused_1, unused_2, unused_3 = pyrcel.binned_activation(S0, ptrace['T'].iloc[n],
                                          aer_out[mode_key].iloc[n], mode_value)
    print(f'{mode_key} activation fraction: {f}')
    activated_amounts.append(f * mode_value.total_N)
    print(f'Activated amount = {f * mode_value.total_N}')

mode1 activation fraction: 1.288261616074726e-07
Activated amount = 0.0005968969479018877
mode2 activation fraction: 4.7332955467824454e-11
Activated amount = 4.550031795934916e-08
mode3 activation fraction: 0.17449272568121385
Activated amount = 247.42725148658394
mode4 activation fraction: 0.9957118369538863
Activated amount = 1.2079633613621483


In [15]:
sum(activated_amounts)

np.float64(248.6358117903943)